# Hub Items Report

## import needed python libraries

In [ ]:
from datetime import datetime
import requests
import pandas as pd

## setup parameters for search request(s)

In [ ]:
group_id = '<group id>'

# url something like 'https://www.arcgis.com/sharing/rest/search'
# or something like 'https://<myOrganizationName>.maps.arcgis.com/sharing/rest/search'
url = 'https://www.arcgis.com/sharing/rest/search'

token = '<token>'

params = {
    'f': 'json',
    'q': '(group:{})'.format(group_id),
    'start': 1,
    'num': 100,
    'token' : token
}

## make request(s) to get all the items shared and add a formatted date column

In [ ]:
# url something like 'https://<myOrganizationName>.opendata.arcgis.com/items'
open_data_url = 'https://<myOrganizationName>.opendata.arcgis.com/items'

r = requests.get(url, params).json()

next_start = r['nextStart']

items = r['results']

total_results = len(items)

results = []

for item in items:
    item['created_dte'] = datetime.utcfromtimestamp((item['created'])/1000).strftime('%Y-%m-%d %H:%M:%S')
    item['modified_dte'] = datetime.utcfromtimestamp((item['modified'])/1000).strftime('%Y-%m-%d %H:%M:%S')
    item['link_url'] = '{}/{}'.format(open_data_url, item['id'])
    results.append(item)    

while next_start != -1:
    params['start'] = next_start
    
    r = requests.get(url, params).json()
    
    items = r['results']
    total_results += len(items)
    
    for item in items:
        item['created_dte'] = datetime.utcfromtimestamp((item['created'])/1000).strftime('%Y-%m-%d %H:%M:%S')
        item['modified_dte'] = datetime.utcfromtimestamp((item['modified'])/1000).strftime('%Y-%m-%d %H:%M:%S')
        item['link_url'] = '{}/{}'.format(open_data_url, item['id'])
        results.append(item)        
    
    next_start = r['nextStart']
    
print ('done')

## create a pandas data frame

In [ ]:
columns = ['title', 'owner', 'type', 'numViews', 'tags', 'created_dte', 'modified_dte', 'link_url']
df = pd.DataFrame(results, columns=columns)

## export to csv

In [ ]:
file_name = 'ngse-hub-item-report-9-20-2018'
df.to_csv('{}.csv'.format(file_name))